# Design How to Combine the data sources
we should have a data source with harmonized ids and meta for each audio file as well as labels 

In [83]:
import pandas as pd
import fma_modules.utils as fma_utils
import librosa


In [70]:
class DataSource():
    def __init__(self,metadata_path, audio_path):
        self.metadata_path = metadata_path 
        self.audio_path = audio_path
        self.columns = ['dataset',
                        'audio_path',
                        'label',
                        'fma_genre_top',
                        'fma_genres',
                        'fma_genres_all']
        return 
    
    def get_file_meta(self):
        return 
    
    def get_audio_paths(self):
        return

        
class FreeMusicArchive(DataSource):
    def __init__(self, metadata_path, audio_path):
        DataSource.__init__(self, metadata_path, audio_path)
        self.tracks = tracks =fma_utils.load(self.metadata_path + 'tracks.csv')

    def get_file_meta(self):
        track_meta = self.tracks['track']
        id_and_labels = (track_meta[['genre_top','genres','genres_all']]
                         .rename(columns={'genre_top': 'fma_genre_top',
                                          'genres': 'fma_genres',
                                          'genres_all': 'fma_genres_all'
                                          
                                          })
                         )
        id_and_labels['dataset']= 'fma'
        id_and_labels['audio_path'] = self.get_audio_paths()
        id_and_labels['label'] = id_and_labels['fma_genre_top']

        
        return id_and_labels[self.columns]
    
    def get_audio_paths(self):
        return (self.tracks.index
                    .to_series()
                    .map(lambda index: fma_utils.get_audio_path(self.audio_path, index))
                )


class GTZAN(DataSource):
    def __init__(self, metadata_path, audio_path):
        DataSource.__init__(self, metadata_path, audio_path)
        self.features_30_sec = pd.read_csv(metadata_path+ 'features_30_sec.csv')
        return
    def get_file_meta(self):      
        id_and_labels = self.features_30_sec[['filename','label']].reset_index()
        id_and_labels['track_id'] = id_and_labels['filename']

        id_and_labels['dataset']= 'gtzan'

        id_and_labels['audio_path'] = self.audio_path +'/'+ id_and_labels.label + '/' + id_and_labels.filename

        harmonized = id_and_labels.set_index('track_id')
        harmonized['fma_genre_top'] = 'n/a'
        harmonized['fma_genres'] = 'n/a'
        harmonized['fma_genres_all'] = 'n/a'

        return harmonized[self.columns]
    

class CombinedDataLoader():
    def __init__(self):
        self.FMA_MEATADATA_PATH ="project_data_source/free_music_archive/fma_metadata/"
        self.FMA_AUDIO_PATH = "project_data_source/free_music_archive/fma_small/"
        self.fma = FreeMusicArchive(self.FMA_MEATADATA_PATH,self.FMA_AUDIO_PATH)
        self.GTZAN_MEATADATA_PATH = "project_data_source/gtzan_dataset/Data/"
        self.GTZAN_AUDIO_PATH = "project_data_source/gtzan_dataset/Data/genres_original"
        self.gtzan = GTZAN(self.GTZAN_MEATADATA_PATH,self.GTZAN_AUDIO_PATH)
        return 
    def get_combined_df(self):
        
        return pd.concat([data.get_file_meta() for data in [self.fma,self.gtzan]])
    


    

In [155]:
class AudioFeatureExtractor():
    def __init__(self,source_data):
        self.df = source_data.copy()
        return
    
    def get_audio_data(self,file_name):
        try:
            y, sr = librosa.load(file_name)
        except:
            return 0
        return y, sr
    
    def add_audio_data_to_df(self):
        self.df['audio_data'] = self.df['audio_path'].apply(self.get_audio_data)

        return 
    
    

In [71]:
data = CombinedDataLoader()

In [73]:
combined = data.get_combined_df()

In [78]:
combined.groupby('dataset')['audio_path'].count()

dataset
fma      106574
gtzan      1000
Name: audio_path, dtype: int64

In [165]:
test_data = combined.iloc[0:10000]


In [166]:
afe = AudioFeatureExtractor(test_data)

In [141]:
afe.df

,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all
track_id,,,,,,
2,fma,project_data_source/free_music_archive/fma_sma...,Hip-Hop,Hip-Hop,[21],[21]
3,fma,project_data_source/free_music_archive/fma_sma...,Hip-Hop,Hip-Hop,[21],[21]
5,fma,project_data_source/free_music_archive/fma_sma...,Hip-Hop,Hip-Hop,[21],[21]
10,fma,project_data_source/free_music_archive/fma_sma...,Pop,Pop,[10],[10]
20,fma,project_data_source/free_music_archive/fma_sma...,NaN,NaN,"[76, 103]","[17, 10, 76, 103]"
...,...,...,...,...,...,...
1270,fma,project_data_source/free_music_archive/fma_sma...,Rock,Rock,[25],"[25, 12]"
1271,fma,project_data_source/free_music_archive/fma_sma...,Rock,Rock,[25],"[25, 12]"
1272,fma,project_data_source/free_music_archive/fma_sma...,Rock,Rock,[25],"[25, 12]"


In [167]:
afe.add_audio_data_to_df()

/var/folders/rt/w6xs9fcn3snbp_cdthc_kjww0000gq/T/ipykernel_5031/209834038.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_name)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!


In [168]:
afe.df.groupby(afe.df['audio_data'] != 0).count()

,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,audio_data
audio_data,,,,,,,
False,9234,9234,6024,6024,9234,9234,9234
True,766,766,766,766,766,766,766


In [154]:
afe.df['audio_data']

track_id
2       ([9.313226e-09, 2.7939677e-09, -3.7252903e-09,...
3                                                ([0], 0)
5       ([-2.561137e-09, 5.5879354e-09, -5.5879354e-09...
10      ([-2.2351742e-08, 7.450581e-09, -7.450581e-09,...
20                                               ([0], 0)
                              ...                        
1270    ([-4.6185278e-14, 2.842171e-13, 6.465939e-13, ...
1271                                             ([0], 0)
1272                                             ([0], 0)
1273                                             ([0], 0)
1274                                             ([0], 0)
Name: audio_data, Length: 1000, dtype: object

In [131]:
afe.df['audio_path'].iloc[1]

'project_data_source/free_music_archive/fma_small/000/000003.mp3'

In [133]:
tid_str = '{:06d}'.format(3)

tid_str[:3]+'/'+tid_str

'000/000003'